In [ ]:
#|default_exp execution_manager

In [ ]:
#|hide
from nblite import nbl_export; nbl_export();

In [ ]:
#|export
from typing import Protocol

In [ ]:
#|export
class Worker(Protocol):
    pass


```python
em.run(worker_id, func_name)
```

If it is in the main process, it just uses func_runner to run it.

If it is not, sends a signal to the worker func.

Maintains an interal register of the queues of each worker, and what each worker is currently running.

In [ ]:
async def func_runner():
    """
    Runs a function. Intercepts the print and redirects it to a callback

    Should possibly also intercept execptions and return them. Should return whether the function sucessfully finished.

    Callback when it is done I guess as well. To be able to unregister it.
    """

In [ ]:
def worker_func(channel, worker_id):
    pass

In [ ]:
async def async_worker_func(channel, worker_id):
    pass

In [ ]:
from netrun.rpc.aio import create_async_channel_pair

channel_a, channel_b = create_async_channel_pair()

await channel_a.send("greeting", "Hello from A!")
key, data = await channel_b.recv()
print(f"B received: {key} = {data}")

B received: greeting = Hello from A!


In [ ]:
async def example_basic_communication():
    """Basic example: two tasks exchanging messages."""
    print("=" * 50)
    print("Example 1: Basic Communication")
    print("=" * 50)

    # Create a pair of connected channels
    channel_a, channel_b = create_async_channel_pair()

    # Send from A, receive on B
    await channel_a.send("greeting", "Hello from A!")
    key, data = await channel_b.recv()
    print(f"B received: {key} = {data}")

    # Send from B, receive on A
    await channel_b.send("response", "Hello back from B!")
    key, data = await channel_a.recv()
    print(f"A received: {key} = {data}")

    # Clean up
    await channel_a.close()
    await channel_b.close()

    print("Done!\n")